In [1]:
import requests

# Fetch public IP
response = requests.get("http://checkip.amazonaws.com")
if response.status_code == 200:
    public_ip = response.text.strip()
    print(f"Public IP: {public_ip}")
else:
    print(f"Failed to fetch public IP. Status code: {response.status_code}")

# The public IP should be in the whitelist of the AWS server, contact Sabah Hussain for access 

Public IP: 104.251.123.179


In [2]:
from PointSpread import get_quote_data, PointSpreadDisplay

In [3]:
dataInput = get_quote_data("2025-01-08", "2025-01-09","XAU/USD")

Transformed symbol: XAUUSD
Date range: ['2025-01-08', '2025-01-09']
Loading local file: Data/XAUUSD_2025-01-08.pkl
Loading local file: Data/XAUUSD_2025-01-09.pkl
All local data files found. Loading data from local storage.
Loaded 27737533 rows from local files.


In [ ]:
# dataInput = get_quote_data("2025-01-08", "2025-01-09","XAU/USD")
result = PointSpreadDisplay(
            df_input=dataInput,
            trade_vol=300,
            date_range=("2025-01-08", "2025-01-09"),
            maker_id="Britannia",
            top_of_book=False,
            symbol="XAU/USD"
        )
